# Advanced Data Mining

## Data preprocessing

### Reducing dataset size

In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import os.path

import pandas as pd

from config import DATASET_SIZE
from scripts.colors import bold, error, success, warning
from scripts.utils import checkpoint, setup

Matplotlib created a temporary cache directory at /tmp/matplotlib-xa8u9ko9 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
setup()

In [4]:
pickle_path = "data/filtered.pkl"

if not os.path.exists(pickle_path):
    # Load dataset from source
    warning("Local dataset not found")
    print("Reading dataset from Postgres... ", end="")
    conn = "postgresql://postgres:adm@db:5432/adm"
    query = "SELECT * FROM filtered"
    df = pd.read_sql(query, conn)
    success("OK")

    # Cache dataset on local machine
    print("Saving dataset to disk... ", end="")
    df.to_pickle(pickle_path)
    success("OK")
else:
    print("Dataset found, reading from disk... ", end="")
    # Load cached dataset
    df = pd.read_pickle(pickle_path)
    success("OK")

rows, cols = df.shape
print("Dataframe contains", bold(f"{rows} rows"), "and", bold(f"{cols} columns"))

Dataset found, reading from disk... OK
Dataframe contains 1612409 rows and 19 columns


Before doing anything, remove duplicates within the same subreddit

In [5]:
rows_count = df.shape[0]
df = df.drop_duplicates(subset=['body', 'subreddit'])
rows_count = rows_count - df.shape[0]
print(rows_count, "row" if rows_count == 1 else "rows", "affected")

rows, cols = df.shape
print("Dataframe contains", bold(f"{rows} rows"), "and", bold(f"{cols} columns"))

7698 rows affected
Dataframe contains 1604711 rows and 19 columns


Reduce dataset to several thousand samples

In [6]:
print("Reducing dataset to", bold(f"{DATASET_SIZE} rows"))  # change in config.py

Reducing dataset to 20000 rows


In [7]:
if DATASET_SIZE <= 0:
    error("Invalid dataset size")

n_classes = len(df['subreddit'].unique())
class_size = DATASET_SIZE // n_classes

From every class, take `class_size` elements - balance dataset

In [8]:
class_samples = []

for subreddit in df['subreddit'].unique():
    _df = df[df['subreddit'] == subreddit].sample(class_size)
    class_samples.append(_df)

df = pd.concat(class_samples).reset_index(drop=True)

print(df['subreddit'].value_counts())
print()

rows, cols = df.shape
print("Dataframe contains", bold(f"{rows} rows"), "and", bold(f"{cols} columns"))

subreddit
gaming         4000
politics       4000
technology     4000
science        4000
programming    4000
Name: count, dtype: int64

Dataframe contains 20000 rows and 19 columns


In [9]:
_ = checkpoint("01-balanced", dataframe=df)